# LangChain的Runnable对象
Runnable是LangChain中最核心的接口，它为所有可执行组件提供统一的抽象层。在LangChain表达式语言(LCEL)中，Runnable是构建处理流程的基础构件。

## 核心特性
1. 统一接口: 所有Runnable对象都支持相同的API方法，包括invoke、stream、batch等
2. 可组合性: 可以使用|操作符将多个Runnable组件链接在一起
3. 异步支持: 支持ainvoke、astream等异步方法
4. 流式处理: 通过stream方法支持流式输出

## 主要Runnable类型
1. RunnableLambda
用于将普通Python函数包装为Runnable对象:

In [24]:
from langchain_core.runnables import RunnableLambda

def func(input):
    print(f"{input},函数调用")

# 将函数包装为Runnable对象
runnableLambda = RunnableLambda(func)

runnableLambda.invoke("RunnableLambda")

RunnableLambda,函数调用


2. RunnablePassthrough
直接传递输入到下一个组件，或用于合并数据:

In [25]:
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough() |  # 传递原始输入
    runnableLambda           # 处理输入
)
chain.invoke("RunnablePassthrough")

RunnablePassthrough,函数调用


3. RunnableParallel 并发调用Runnable，并为每个Runnable提供相同的输入。

In [36]:
from langchain_core.runnables import RunnableParallel

def func1(input):
    print(f"\n{input},func1函数调用")
    return input
def func2(input):
    print(f"\n{input},func2函数调用")
    return input
def func3(input):
    print(f"\n{input},func3函数调用")
    return input

# # 方式一
# chain = RunnableParallel(
#     {
#         "func1": RunnableLambda(func1), 
#         "func2": RunnableLambda(func2), 
#         "func3": RunnableLambda(func3)
#     }
# )

# chain.invoke(1)

# # 方式二
# chain = RunnableParallel(
#     func1 = RunnableLambda(func1), 
#     func2 = RunnableLambda(func2), 
#     func3 = RunnableLambda(func3)
# )

# chain.invoke(1)

# 方式三
chain = (
    RunnablePassthrough() |
    {
        "func1": RunnableLambda(func1), 
        "func2": RunnableLambda(func2), 
        "func3": RunnableLambda(func3)
    }
)

chain.invoke({"a": 1})





{'a': 1},func1函数调用

{'a': 1},func2函数调用

{'a': 1},func3函数调用


{'func1': {'a': 1}, 'func2': {'a': 1}, 'func3': {'a': 1}}

4. RunnableGenerator 用于流式处理和生成数据:

In [41]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableGenerator

def streaming_parse(chunks):
    for chunk in chunks:
        yield chunk.content

streaming_output = RunnableGenerator(streaming_parse)

llm = ChatOpenAI(
    model="qwen-turbo", 
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming=True
)

chain = llm | streaming_output

for chunk in chain.stream("介绍一下你自己"):
    print(chunk, end="", flush=True)

你好！我是Qwen，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够帮助你回答问题、创作文字，如写故事、公文、技术文档等，还能进行逻辑推理、编程，表达观点，玩游戏等。我的训练数据非常丰富，因此可以支持多种语言和领域的问题。

如果你有任何问题或需要帮助，欢迎随时告诉我！我会尽力为你提供有用的信息和建议。😊

4.1 自定义流生成器 基于流式生成的回答来自定义一个输出解析器

In [25]:
import asyncio
import os
import time
from typing import AsyncIterator, Iterator, List
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableGenerator
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

# 基础流式生成器
def streaming_parse(chunks):
    for chunk in chunks:
        yield chunk.content

streaming_output = RunnableGenerator(streaming_parse)

# 提示词
prompt = ChatPromptTemplate.from_template(
    """
    回答以CSV的格式返回中文列表，不要返回其他内容。
    输出10个与{transportation}类似的交通工具
    """
)

# 大模型
llm = ChatOpenAI(
    model="qwen-turbo", 
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming=True
)

print('=' * 20)

chain = prompt | llm | streaming_output

for chunk in chain.stream({"transportation" : "飞机"}):
    print(chunk, end="", flush=True)
    
print('\n')
print('=' * 20)

# 自定义解析器，用于拆分llm令牌的迭代器
# 将用逗号分割字符串，把元素放入列表中
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    #保留输入，直到逗号才进行操作
    buffer = ''
    for chunk in input:
        # 将当前块添加到缓冲区
        buffer += chunk
        while ',' in buffer:
            comma_index = buffer.index(',')
            print('\n返回一个块')
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1:]
    print('\n最后一个块')
    yield[buffer.strip()]
    
list_chain = (prompt | llm | StrOutputParser() | split_into_list)

for chunk in list_chain.stream({"transportation" : "飞机"}):
    print(chunk, end="", flush=True)
    

print('\n')
print('=' * 20)

# 添加异步操作
async def asplit_into_list(input: AsyncIterator[str]) -> AsyncIterator[List[str]]:
    #保留输入，直到逗号才进行操作
    buffer = ''
    async for chunk in input:
        # 将当前块添加到缓冲区
        buffer += chunk
        while ',' in buffer:
            comma_index = buffer.index(',')
            print('\n异步返回一个块')
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1:]
    print('\n异步返回最后一个块')
    yield[buffer.strip()]


alist_chain = (prompt | llm | StrOutputParser() | asplit_into_list)

async def run_async_chain():
    print("开始执行异步链...")
    async for chunk in alist_chain.astream({"transportation": "飞机"}):
        print(f"异步链输出: {chunk}", end="", flush=True)
        await asyncio.sleep(1)  # 模拟异步处理时间
    print("\n异步链执行完成")
    
async def main():
    # 启动异步链但不等待它完成
    async_task = asyncio.create_task(run_async_chain())
    
    # 在异步任务运行的同时执行同步逻辑
    for i in range(10):
        print(f"同步逻辑：打印{i}")
        await asyncio.sleep(0.5)  # 使用异步延迟
    
    # 如果需要等待异步任务完成再结束程序
    await async_task

await main()

飞机,直升机,滑翔机,热气球,飞艇,无人机,航天飞机,螺旋桨飞机,喷气式飞机,超音速飞机


返回一个块
['飞机']
返回一个块
['直升机']
返回一个块
['热气球']
返回一个块
['飞艇']
返回一个块
['滑翔机']
返回一个块
['无人机']
返回一个块
['喷气式公务机']
返回一个块
['螺旋桨飞机']
返回一个块
['战斗机']
最后一个块
['航天飞机']

同步逻辑：打印0
开始执行异步链...

异步返回一个块
异步链输出: ['飞机']同步逻辑：打印1
同步逻辑：打印2

异步返回一个块
异步链输出: ['直升机']同步逻辑：打印3
同步逻辑：打印4

异步返回一个块
异步链输出: ['热气球']同步逻辑：打印5
同步逻辑：打印6

异步返回一个块
异步链输出: ['飞艇']同步逻辑：打印7
同步逻辑：打印8

异步返回一个块
异步链输出: ['滑翔机']同步逻辑：打印9

异步返回一个块
异步链输出: ['无人机']
异步返回一个块
异步链输出: ['航天飞机']
异步返回一个块
异步链输出: ['喷气式战斗机']
异步返回一个块
异步链输出: ['螺旋桨飞机']
异步返回最后一个块
异步链输出: ['超音速飞机']
异步链执行完成


4. RunnableSequence
将多个Runnable组合为一个执行序列:

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("{location}在哪个省")

llm = ChatOpenAI(
    model="qwen-turbo", 
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming=True
)

output_parser = StrOutputParser()

# 显式创建
sequence = RunnableSequence(
    first=prompt_template,
    middle=[llm],
    last=output_parser
)

# 更常见的是使用管道操作符
sequence = prompt_template | llm | output_parser

result = sequence.invoke({"location": "郑州"})

print(result)

郑州是中国河南省的省会城市。河南位于中国中部，是中华民族的重要发祥地之一，拥有丰富的历史文化和自然景观。郑州作为河南省的省会，是中国重要的综合交通枢纽和中原经济区的核心城市。


5. 使用@Chain装饰器 可以将任意函数变成链:

In [55]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="qwen-turbo", 
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming=True
)

@chain
def custom_chain(input):
    prompt_template1 = ChatPromptTemplate.from_template("给我介绍一下{location}")
    prompt_template2 = ChatPromptTemplate.from_template("{intruduction}\n\n总结上面的内容，精简到20字以内")
    chain = (
        prompt_template1 
        | llm 
        | StrOutputParser() 
        |{ "intruduction": RunnablePassthrough() }
        | prompt_template2 
        | llm 
        | StrOutputParser()
    )
    return chain.invoke(input)

custom_chain.invoke({"location": "郑州"})

'郑州：华夏文明发源地，现代交通枢纽与文化名城。'

## 调用方法
Runnable对象提供多种调用方式:

In [ ]:
# 同步调用
result = chain.invoke({"input": "query"})

# 流式调用
for chunk in chain.stream({"input": "query"}):
    print(chunk, end="", flush=True)

# 批量处理
results = chain.batch([{"input": "query1"}, {"input": "query2"}])

# 异步调用
result = await chain.ainvoke({"input": "query"})

## 配置与回调
Runnable支持运行时配置:

In [11]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(
    model="qwen-turbo", 
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
).configurable_fields(
    temperature=ConfigurableField(
        id="temperature",  # 配置标识符
        name="Temperature",
        description="Controls randomness of the output. Higher values mean more randomness."
    )
)

prompt = """
你是一位创意作家。请为一款名为'星际迷航者'的科幻游戏创作一个主角背景故事。
这个角色是一位来自未来的太空探险家，拥有特殊能力。
请详细描述角色的外貌、性格、特殊能力和人生经历。字数在200字左右。
"""

# 使用低温度运行 - 更确定性的输出
result1 = llm.invoke(prompt, config={
    "configurable": {
        "temperature": 0.2
    }
})

print("使用低温度运行 - 更确定性的输出\n\n",result1)

# 使用高温度运行 - 更创造性和多样化的输出
result2 = llm.invoke(prompt, config={
    "configurable": {
        "temperature": 1.2
    }
})

print("\n\使用高温度运行 - 更创造性和多样化的输出\n\n",result2)

# 配置回调
result3 = llm.invoke("你好", config={"callbacks": [StdOutCallbackHandler()]})

使用低温度运行 - 更确定性的输出

 content='主角艾瑞克·星河，来自公元3125年，是一名天赋异禀的太空探险家。他身高1.9米，皮肤呈淡金色，双眼如星辰般闪烁着幽蓝光芒，额前有一道神秘的银色纹路，仿佛连接着宇宙的秘密。性格冷静而坚韧，内心却充满对未知的好奇与热情。\n\n艾瑞克的能力源于基因改造与量子意识融合，他能感知空间波动并预判航线，甚至短暂操控引力场。他曾是地球联邦最年轻的星际舰长，因一次意外穿越虫洞，失去了挚爱的舰队和家园。这次经历让他更加坚定探索宇宙的使命，寻找新的栖息地以避免人类灭绝的命运。他的旅程不仅是冒险，更是对自我救赎的追寻。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 69, 'total_tokens': 236, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-turbo', 'system_fingerprint': None, 'id': 'chatcmpl-c41e9488-816f-9d23-ae7a-7835a60d36b5', 'finish_reason': 'stop', 'logprobs': None} id='run-0ace3bb1-3ffe-4384-aa7d-c20e1fd8a5b5-0' usage_metadata={'input_tokens': 69, 'output_tokens': 167, 'total_tokens': 236, 'input_token_details': {}, 'output_token_details': {}}

\使用高温度运行 - 更创造性和多样化的输出

 content='主角名为凯洛·星影，是一位来自公元3125年的太空探险家。他高挑修长，皮肤泛着微光的银灰色，双眼如深邃的黑洞，能散发蓝紫色光芒。冷静且富有智慧，他总能在危机中找到转机。\n\n凯洛拥有一种罕见的“时空共鸣”能力，能够感知过去与未来的一丝涟漪，并短暂操控时间流速。他的特殊技能源于一